In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

from pyspark.sql.functions import col,desc,to_date, coalesce,filter,count,avg

In [ ]:
df = spark.read.option('header','true').csv('../../data/prueba')

In [ ]:
df = df.withColumn(
    'fecha',
    coalesce(
        to_date(col("fecha"), "dd/MM/yyyy"),
        to_date(col("fecha"), "yyyy-MM-dd")
    )
)

In [ ]:
df.show(10)
df.printSchema()


In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType,DateType

In [ ]:
df_transformado = df.selectExpr(
 'cast(id_transaccion as String) as id_transaccion',
 'cast(fecha as Date) as fecha',
 'cast(id_cliente as string) as id_cliente',
 'cast(id_producto as string) as id_producto',
 'cast(categoria as string) as categoria',
 'cast(subcategoria as string) as subcategoria',
 'cast(precio as Double) as precio',
 'cast(cantidad as Integer)as cantidad',
 'cast(total as Double)as total',
 'cast(ubicacion_cliente as string) as ubicacion_cliente',
 'cast(metodo_pago as string) as metodo_pago',
 'cast(estado_pedido as string) as estado_pedido',
)


In [ ]:
df_transformado.count()

In [ ]:
df_sinDup = df_transformado.dropDuplicates()

df_sinDup.printSchema()


In [ ]:
df_sinDup.printSchema()

In [22]:
# ¿Cuáles son las 3 categorías de productos más vendidas por ubicación?

# GROUPBY(UBICACION)
# COUNT(CATEGORIAS)

df_conteo = df_sinDup.groupBy("ubicacion_cliente", "categoria") \
    .agg(count("*").alias("cantidad")).orderBy(desc("ubicacion_cliente")).limit(3).show()

# ¿Cuál es el valor promedio de transacción por método de pago?

# GROUPBY METODO DE PAGO
# AVR(TRANSACCION)

df_sinDup.groupBy('metodo_pago').agg(
    avg('total').alias('PROMEDIO_TRANSACCION')
).orderBy('PROMEDIO_TRANSACCION').show()

# ¿Existe alguna tendencia notable en las ventas por día de la semana?



+-----------------+---------+--------+
|ubicacion_cliente|categoria|cantidad|
+-----------------+---------+--------+
|         Trujillo| Juguetes|      17|
|         Trujillo|     NULL|       7|
|         Trujillo| Deportes|      29|
+-----------------+---------+--------+

+-----------------+--------------------+
|      metodo_pago|PROMEDIO_TRANSACCION|
+-----------------+--------------------+
|  Tarjeta Débito |              588.12|
|          PayPal |   659.7174999999999|
|        Efectivo |   689.5986666666668|
|    Transferencia|   740.4325176470588|
|           PayPal|   751.6330112994352|
|         Efectivo|   780.3083532608697|
|   Tarjeta Débito|   801.0704300518139|
|  Tarjeta Crédito|   835.6519837837837|
| Tarjeta Crédito |   878.0018666666668|
|   Transferencia |   886.8972857142859|
+-----------------+--------------------+



In [23]:
df_sinDup.printSchema()

root
 |-- id_transaccion: string (nullable = true)
 |-- fecha: date (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- id_producto: string (nullable = true)
 |-- categoria: string (nullable = true)
 |-- subcategoria: string (nullable = true)
 |-- precio: double (nullable = true)
 |-- cantidad: integer (nullable = true)
 |-- total: double (nullable = true)
 |-- ubicacion_cliente: string (nullable = true)
 |-- metodo_pago: string (nullable = true)
 |-- estado_pedido: string (nullable = true)



In [30]:
# ¿Existe alguna tendencia notable en las ventas por día de la semana?

df_sinDup.groupBy('fecha').agg(
    count('id_transaccion').alias('Cantidad')
).orderBy(desc('fecha')).show()



+----------+--------+
|     fecha|Cantidad|
+----------+--------+
|2025-05-02|       3|
|2025-05-01|       4|
|2025-04-30|       1|
|2025-04-29|       3|
|2025-04-28|       1|
|2025-04-27|       4|
|2025-04-26|       5|
|2025-04-25|       4|
|2025-04-24|       3|
|2025-04-23|       2|
|2025-04-22|       5|
|2025-04-21|       3|
|2025-04-20|      11|
|2025-04-19|       4|
|2025-04-18|       2|
|2025-04-17|       1|
|2025-04-16|       2|
|2025-04-15|       2|
|2025-04-14|       5|
|2025-04-13|       9|
+----------+--------+
only showing top 20 rows

